# Week 10 Live Coding Demo — SVD, Eigenpairs, Interpolation, Taylor, Root Finding
PHYS 77 — designed for first-time learners. Every cell explains **what** and **why**.


## Part 1 — SVD: step-by-step action on a colored unit circle

In [ ]:
# GOAL:
# Visualize how a 2x2 real (non-symmetric) matrix A transforms vectors by breaking A into SVD: A = U Σ V^T.
# We'll act on a colored unit circle (many points). The color encodes the original angle (phase) so we can track points.
#
# STEP 0 (setup): define A, compute its SVD, and generate a colored unit circle.
import numpy as np, matplotlib.pyplot as plt

# A non-symmetric real matrix (not a pure rotation or reflection)
A = np.array([[1.2, 0.6],
              [0.1, 0.9]])

# SVD: A = U Σ V^T
U, S, VT = np.linalg.svd(A)   # U: 2x2, S: length-2 vector, VT: 2x2

# A dense set of points on the unit circle (for a smooth curve)
phi = np.linspace(0, 2*np.pi, 600, endpoint=False)  # angles 0..2π
circle = np.vstack([np.cos(phi), np.sin(phi)])       # shape (2, 600)

print("Matrix A =\n", A)
print("Singular values S =", S)   # non-negative scale factors
print("Check orthonormal: U^T U=I ?", np.allclose(U.T @ U, np.eye(2)))  # -> True
print("Check orthonormal: V^T V=I ?", np.allclose(VT.T @ VT, np.eye(2)))# -> True

# Plot the original colored unit circle
plt.figure(figsize=(5,4.5))
plt.scatter(circle[0], circle[1], c=phi, s=6, cmap='viridis')
plt.axis('equal'); plt.title("Original unit circle (color = angle)")
plt.xlabel("x"); plt.ylabel("y"); plt.colorbar(label="angle (rad)")
plt.tight_layout(); plt.show()


In [ ]:
# STEP 1: Apply V^T to the circle (first step in A = U Σ V^T).
# Intuition: V^T is a rotation/reflection in input space that aligns the special axes with the coordinate axes.
X1 = VT @ circle   # shape (2, 600)

plt.figure(figsize=(5,4.5))
plt.scatter(X1[0], X1[1], c=phi, s=6, cmap='viridis')
plt.axis('equal'); plt.title("After V^T (input axes aligned)")
plt.xlabel("x"); plt.ylabel("y"); plt.colorbar(label="original angle (rad)")
plt.tight_layout(); plt.show()


In [ ]:
# STEP 2: Apply Σ (sigma) next: Σ scales the aligned axes by singular values S[0], S[1].
# Convert S (vector) into a diagonal scaling matrix.
Sigma = np.diag(S)
X2 = Sigma @ X1

plt.figure(figsize=(5,4.5))
plt.scatter(X2[0], X2[1], c=phi, s=6, cmap='viridis')
plt.axis('equal'); plt.title("After Σ V^T (axes scaled → ellipse)")
plt.xlabel("x"); plt.ylabel("y"); plt.colorbar(label="original angle (rad)")
plt.tight_layout(); plt.show()


In [ ]:
# STEP 3: Apply U last: rotates the scaled ellipse into its final orientation in output space.
X3 = U @ X2   # this equals A @ circle

plt.figure(figsize=(5,4.5))
plt.scatter(X3[0], X3[1], c=phi, s=6, cmap='viridis')
plt.axis('equal'); plt.title("After U Σ V^T = A (final result)")
plt.xlabel("x"); plt.ylabel("y"); plt.colorbar(label="original angle (rad)")
plt.tight_layout(); plt.show()


In [ ]:
# SANITY CHECK: Direct multiplication A @ circle matches U Σ V^T @ circle.
direct = A @ circle
print("Max absolute difference between direct and staged:", np.max(np.abs(direct - X3)))  # ~1e-15


## Part 2 — Eigenpairs: 3 masses connected by 4 springs (fixed ends)

In [ ]:
# PHYSICAL MODEL (small oscillations):
# wall -- k0 -- m1 -- k1 -- m2 -- k2 -- m3 -- k3 -- wall
# Let y1, y2, y3 be small displacements of the three masses from equilibrium.
# Hooke's law: force = -k * extension. For each mass, sum of spring forces = m * y_ddot.
# In matrix form with unit masses (M = I):  y_ddot = -K y,  where K is the stiffness matrix.
#
# BUILD K from Hooke's law (units abstracted for demo):
import numpy as np

k0, k1, k2, k3 = 2.0, 1.0, 1.5, 3.0   # spring constants (>0)
# For fixed ends: diagonal entries add the adjacent spring constants; off-diagonals are -spring between neighbors.
K = np.array([[k0 + k1,    -k1,          0. ],
              [   -k1,  k1 + k2,       -k2 ],
              [    0.,     -k2,     k2 + k3]], dtype=float)

print("Stiffness matrix K =\n", K)
print("Symmetric?", np.allclose(K, K.T))  # -> True
# K is symmetric positive definite for positive spring constants → nice eigenproperties.


In [ ]:
# EIGEN-DECOMPOSITION (modes and squared frequencies):
# Solve K v = λ v where λ = ω^2 if masses are 1 kg. Then natural frequencies are ω = sqrt(λ).
import numpy as np, matplotlib.pyplot as plt

lam, V = np.linalg.eigh(K)         # symmetric → real eigenvalues, orthonormal eigenvectors
omega = np.sqrt(lam)               # natural frequencies
print("Eigenvalues (λ = ω^2):", lam)
print("Frequencies ω:", omega)

# Plot the mode shapes (eigenvectors) as vertical deflections for masses [m1,m2,m3].
xpos = np.arange(1, 4)  # positions of masses along a line
plt.figure(figsize=(6,3.2))
for i in range(3):
    plt.plot(xpos, V[:,i], 'o-', label=f"mode {i+1}, ω≈{omega[i]:.3f}")
plt.axhline(0, color='k', lw=0.8); plt.xticks([1,2,3], ["m1","m2","m3"])
plt.ylabel("relative displacement (arbitrary)")
plt.title("Normal modes of 3-mass chain")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# SVD VS EIGEN FOR SPD:
# For symmetric positive definite K, singular values equal eigenvalues, and left/right singular vectors match (up to sign).
U_K, S_K, VT_K = np.linalg.svd(K)
print("eigvals λ   :", lam)      # sorted ascending
print("singular vals:", S_K)     # equal to λ (for SPD), may appear reverse-sorted
print("U ≈ V ?     :", np.allclose(U_K, VT_K.T))  # -> True up to column signs
# Note: Columns of V (from eigh) and U (from svd) can differ by sign; that is physically irrelevant for a mode shape.


## Part 3 — Interpolation: nearest, linear, cubic spline, Lagrange, Newton

In [ ]:
# DATA SETUP:
# Suppose we measured a detector response y at non-uniform x locations.
# We want to estimate y at in-between x values (interpolation).
import numpy as np, matplotlib.pyplot as plt

x = np.array([0.0, 0.8, 1.5, 2.2, 3.0, 3.7, 4.1, 5.0])
y = np.array([0.1, 0.9, 0.2, 1.6, 1.2, 1.9, 1.4, 0.7])
xx = np.linspace(-0.2, 5.2, 400)

plt.figure(figsize=(5.5,3.5))
plt.plot(x, y, 'ko', label="data")
plt.title("Sampled data (non-uniform x)")
plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# NEAREST-NEIGHBOR INTERPOLATION (baseline; fast but blocky)
import numpy as np, matplotlib.pyplot as plt

# For each xx point, find the index of the closest x sample (argmin of absolute difference)
nearest_idx = np.abs(xx[:,None] - x[None,:]).argmin(axis=1)
yy_nearest = y[nearest_idx]

plt.figure(figsize=(6,3.5))
plt.step(xx, yy_nearest, where='mid', label='nearest')
plt.plot(x, y, 'ko', label='data')
plt.title("Nearest-neighbor interpolation (blocky)")
plt.xlabel("x"); plt.ylabel("y")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# LINEAR INTERPOLATION (piecewise straight lines; continuous, but corners at data points)
import numpy as np, matplotlib.pyplot as plt

yy_linear = np.interp(xx, x, y)  # NumPy provides simple 1-D linear interpolation
plt.figure(figsize=(6,3.5))
plt.plot(xx, yy_linear, '-', label='linear')
plt.plot(x, y, 'ko', label='data')
plt.title("Linear interpolation")
plt.xlabel("x"); plt.ylabel("y")
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# CUBIC SPLINE INTERPOLATION (smooth first & second derivatives)
import numpy as np, matplotlib.pyplot as plt
try:
    from scipy.interpolate import CubicSpline
    cs = CubicSpline(x, y, bc_type='natural')  # natural spline: second derivative zero at ends
    yy_spline = cs(xx)
    plt.figure(figsize=(6,3.5))
    plt.plot(xx, yy_spline, '-', label='cubic spline')
    plt.plot(x, y, 'ko', label='data')
    plt.title("Cubic spline interpolation (natural)")
    plt.xlabel("x"); plt.ylabel("y")
    plt.legend(); plt.tight_layout(); plt.show()
except Exception as e:
    print("CubicSpline requires SciPy. If missing, install scipy. Skipping spline demo.\n", e)


In [ ]:
# LAGRANGE POLYNOMIAL INTERPOLATION (global polynomial through all points)
# We'll implement a simple Lagrange evaluator so we don't rely on SciPy.
import numpy as np, matplotlib.pyplot as plt

def lagrange_interp(x_nodes, y_nodes, x_eval):
    x_nodes = np.asarray(x_nodes); y_nodes = np.asarray(y_nodes)
    x_eval = np.asarray(x_eval)
    n = len(x_nodes)
    yy = np.zeros_like(x_eval, float)
    for k in range(n):
        # L_k(x) = Π_{j != k} (x - x_j)/(x_k - x_j)
        Lk = np.ones_like(x_eval, float)
        for j in range(n):
            if j != k:
                Lk *= (x_eval - x_nodes[j])/(x_nodes[k] - x_nodes[j])
        yy += y_nodes[k] * Lk
    return yy

yy_lagr = lagrange_interp(x, y, xx)
plt.figure(figsize=(6,3.5))
plt.plot(xx, yy_lagr, '-', label='Lagrange polynomial')
plt.plot(x, y, 'ko', label='data')
plt.title("Lagrange interpolation (global polynomial)")
plt.xlabel("x"); plt.ylabel("y")
plt.ylim([-2, 3])
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# NEWTON'S DIVIDED-DIFFERENCE POLYNOMIAL (algebraically same as Lagrange; easier to update)
import numpy as np, matplotlib.pyplot as plt

def divided_differences(x_nodes, y_nodes):
    # Build the divided-difference table and return top row (coefficients a_k).
    x_nodes = np.asarray(x_nodes, dtype=float)
    y_nodes = np.asarray(y_nodes, dtype=float)
    n = len(x_nodes)
    table = np.zeros((n, n), dtype=float)
    table[:,0] = y_nodes
    for j in range(1, n):
        for i in range(n-j):
            table[i,j] = (table[i+1,j-1] - table[i,j-1]) / (x_nodes[i+j] - x_nodes[i])
    return table[0,:]  # a_0, a_1, ..., a_{n-1}

def newton_eval(x_nodes, a, x_eval):
    # Evaluate p(x) = a0 + a1(x-x0) + a2(x-x0)(x-x1) + ...
    x_nodes = np.asarray(x_nodes, dtype=float)
    x_eval  = np.asarray(x_eval, dtype=float)
    n = len(a)
    out = np.zeros_like(x_eval, dtype=float)
    for m in range(n-1, -1, -1):
        out = a[m] + (x_eval - x_nodes[m]) * out
    return out

a = divided_differences(x, y)
yy_newton = newton_eval(x, a, xx)

# Compare Newton vs Lagrange (should be numerically the same polynomial)
max_diff = float(np.max(np.abs(yy_newton - lagrange_interp(x, y, xx))))
print("Max |Newton - Lagrange| on grid =", max_diff)  # -> tiny (~1e-12 to 1e-9)

plt.figure(figsize=(6,3.5))
plt.plot(xx, yy_newton, '-', label='Newton polynomial')
plt.plot(x, y, 'ko', label='data')
plt.title("Newton interpolation (same poly as Lagrange)")
plt.xlabel("x"); plt.ylabel("y")
plt.ylim([-2, 3])
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
# ADD ONE NEW DATA POINT and update ONLY ONE new Newton term.
import numpy as np, matplotlib.pyplot as plt

# New measurement arrives:
x_new = [4.8, 0.5]
y_new = [0.9, 0.1]

# Extend nodes:
x_aug = np.append(x, x_new)
y_aug = np.append(y, y_new)

# Recompute divided differences efficiently (simple re-run here for clarity).
a_aug = divided_differences(x_aug, y_aug)
yy_aug = newton_eval(x_aug, a_aug, xx)

plt.figure(figsize=(6.2,3.5))
plt.plot(xx, yy_newton, '-', label='before (Newton poly)')
plt.plot(xx, yy_aug,   '--', label='after adding point')
plt.plot(x, y, 'ko'); plt.plot([x_new], [y_new], 'rs', label='new point')
plt.title("Newton polynomial updated with one new data point")
plt.ylim([-2, 3])
plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.tight_layout(); plt.show()


## Part 4 — Taylor series: Gaussian $e^{-x^2}$ around $x=$ 0.5

### Taylor series of \(e^{-x^2}\) about \(a=0.5\)

We use the Taylor expansion
$$
e^{-x^2}=\sum_{n=0}^{\infty}\frac{f^{(n)}(a)}{n!}(x-a)^n, \qquad a=0.5.
$$

For \(f(x)=e^{-x^2}\), the derivatives satisfy
$$
f^{(n)}(x)=(-1)^n\,H_n(x)\,e^{-x^2},
$$
where \(H_n\) are the physicists’ Hermite polynomials,
$$
H_0(x)=1,\qquad H_1(x)=2x,\qquad
H_{n+1}(x)=2x\,H_n(x)-2n\,H_{n-1}(x).
$$

Therefore the Taylor coefficients are
$$
\frac{f^{(n)}(a)}{n!}=(-1)^n\,\frac{H_n(a)}{n!}\,e^{-a^2}.
$$

Because \(a\neq 0\), **all powers** (odd and even) appear in the series.


In [ ]:
import numpy as np, matplotlib.pyplot as plt, math

# Taylor of exp(-x^2) about a=0.5 using Hermite derivatives
def hermite_phys_vals(a, N): 
    """
    Compute the values of physicists’ Hermite polynomials H_0..H_N at the point a.

    Parameters
    ----------
    a : float
        Point at which to evaluate the Hermite polynomials.
    N : int
        Maximum order of the Hermite polynomial.

    Returns
    -------
    H : list of float
        Values of H_0(a), H_1(a), ..., H_N(a).
    """
    H = [1.0, 2.0*a] if N>=1 else [1.0]                  # init
    for n in range(1, N):
        H.append(2*a*H[n] - 2*n*H[n-1])                  # recurrence
    return H[:N+1]

def taylor_exp_neg_x2(x, N, a=0.5):
    """
    Compute the Taylor series of exp(-x^2) about point a to order N using Hermite polynomials.

    Parameters
    ----------
    x : array-like or float
        Points at which to evaluate the Taylor series.
    N : int
        Maximum order of the Taylor polynomial.
    a : float, optional
        Expansion point (default is 0.5).

    Returns
    -------
    out : ndarray or float
        Value(s) of the Taylor series approximation at x.
    """

    H = hermite_phys_vals(a, N)  
    coeff = [((-1)**n)*H[n]*math.exp(-a*a)/math.factorial(n) for n in range(N+1)]
    dx = np.asarray(x) - a
    out = np.zeros_like(dx, dtype=float)
    for n, c in enumerate(coeff):         # sum c_n (x-a)^n
        out += c * dx**n
    return out

# visualize
a, Nmax = 0.5, 6
xx = np.linspace(-0.9, 1.9, 600)
plt.figure(figsize=(6.4,4.9))
for N in [0,1,2,3,4,5]:
    plt.plot(xx, taylor_exp_neg_x2(xx, N, a), label=f"N={N}")  # Taylor N
plt.plot(xx, np.exp(-xx**2), 'k', lw=2, label="exp(-x^2) (true)")  # truth
plt.axvline(a, ls='--', color='gray') ; plt.title("Taylor series of $e^{-x^2}$ about x=0.5: all orders appear")
plt.xlabel("x"); plt.ylabel("y"); plt.legend(loc="best"); plt.tight_layout(); plt.show()

## Part 5 — Root finding: bisection vs Newton vs fixed-point; then library solvers

In [ ]:
import numpy as np, matplotlib.pyplot as plt

# Problem ----------------------------------------------------------------------
f  = lambda x: x**3 - x - 1
fp = lambda x: 3*x**2 - 1 # df/dx

# Bisection that records midpoints --------------------------------------------
def bisection_iters(f, a, b, tol=1e-10, maxit=200):
    if np.sign(f(a)) == np.sign(f(b)): raise ValueError("Need opposite signs")  # sanity
    xs = []
    for _ in range(maxit):
        m = 0.5*(a+b); xs.append(m)                                             # store iterate
        if abs(f(m)) < tol or 0.5*(b-a) < tol: break
        if f(a)*f(m) < 0: b = m
        else: a = m
    return xs

# Newton that records x_k ------------------------------------------------------
def newton_iters(f, fp, x0, tol=1e-12, maxit=50):
    xs, x = [float(x0)], float(x0)
    for _ in range(maxit):
        # fp is the derivative of f with respect to x
        fx, fpx = f(x), fp(x) 
        if fpx == 0: break
        x_new = x - fx/fpx
        xs.append(x_new)                                                        # store iterate
        if abs(x_new - x) < tol: break
        x = x_new
    return xs

# Run both methods -------------------------------------------------------------
xs_bis = bisection_iters(f, 1.0, 2.0)
xs_new = newton_iters(f, fp, x0=1.5)

root_est = xs_bis[-1]                                                           # good reference
print("Bisection root ~", root_est, "  |f| =", abs(f(root_est)))
print("Newton   root ~", xs_new[-1],   "  |f| =", abs(f(xs_new[-1])))

# Plot x vs iteration ----------------------------------------------------------
plt.figure(figsize=(6,3.6))
plt.plot(range(len(xs_bis)), xs_bis, 'o-', label='Bisection x_k', alpha=0.5)               # x_k path
plt.plot(range(len(xs_new)), xs_new, 's--', label='Newton x_k', alpha=0.5)
plt.axhline(root_est, linestyle=':', label='Reference root')
plt.xlabel('Iteration k'); plt.ylabel('x_k'); plt.title('Convergence: x vs iteration')
plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
true = root_est                                                                  # use bisection as truth
ebis = [abs(x-true) for x in xs_bis]
enew = [abs(x-true) for x in xs_new]
plt.figure(figsize=(6,3.6))
plt.semilogy(ebis, 'o-', label='Bisection |x_k - x*|')
plt.semilogy(enew, 's--', label='Newton |x_k - x*|')
plt.xlabel('Iteration k'); plt.ylabel('Error'); plt.title('Error decay (log scale)')
plt.legend(); plt.tight_layout(); plt.show()

In [ ]:
# MULTIPLE ROOTS: show library solvers can find different roots depending on guess/bracket.
# Use f(x) = sin(x) - 0.2 x, which has many roots.
import numpy as np, matplotlib.pyplot as plt
xx = np.linspace(-10, 10, 1000)
f2 = lambda x: np.sin(x) - 0.2*x

plt.figure(figsize=(6.2,3.6))
plt.plot(xx, f2(xx), label="f2(x) = sin(x) - 0.2x")
plt.axhline(0, color='k', lw=0.8)
plt.ylim(-2, 2); plt.title("Multiple roots example")
plt.legend(); plt.tight_layout(); plt.show()

from scipy.optimize import root_scalar, fsolve
# root_scalar with different brackets → different roots
r1 = root_scalar(f2, bracket=(-3, -1)).root
r2 = root_scalar(f2, bracket=(2.5, 10)).root
print("root_scalar bracket [-3, -1]  ->", r1)
print("root_scalar bracket [2.5, 10] ->", r2)

# fsolve uses a starting guess → can land on different roots
x0 = [1.0, 3.0]
s1 = float(fsolve(f2, x0=x0[0])[0])    # near the small positive root
s2 = float(fsolve(f2, x0=x0[1])[0])    # near a larger-x root
print(f"fsolve x0={x0[0]} ->", s1)
print(f"fsolve x0={x0[1]} ->", s2)

In [ ]:
# "WRONG" ROOT DEMO: projectile vertical position y(t)=v0 sinθ t - 1/2 g t^2 has two roots: t=0 and the landing time.
# If you give a starting guess near 0 to a solver that doesn't require a bracket, it may find the trivial root t=0.
import numpy as np

from scipy.optimize import root_scalar, fsolve
v0, g, theta = 20.0, 9.81, np.deg2rad(35)
y = lambda t: v0*np.sin(theta)*t - 0.5*g*t**2

# Good: bracket that avoids t=0 will find the positive landing time.
t_good = root_scalar(y, bracket=(0.1, 5.0)).root
print("Bracketed landing time ~", t_good)  # positive root

# Potentially misleading: initial guess at 0 leads to root at 0.
t0_bad = float(fsolve(y, x0=0.0)[0])   # This returns exactly 0.0 (the launch instant)
print("fsolve with x0=0.0 returned", t0_bad, "(the trivial root)")

# A better starting guess moves to the physical root.
t0_good = float(fsolve(y, x0=3.0)[0])
print("fsolve with x0=3.0 returned", t0_good, "(the landing time)")